In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from __future__ import division
%load_ext autoreload
%autoreload 2
import DCMFlow_093 as dcm
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import OrderedDict
import time
import random
import operator
import random as rn
import os
from datetime import datetime
import math
import copy
import matplotlib.pyplot as plt
import os
import collections
from scipy.stats import truncnorm
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

### Functions to generate nested logit tree and utility expressions

In [3]:
def generate_nltree(strct=None):
    n_levels = len(strct)
    nltree_dict = {'l0':'Root', 'l1':['L1C' + str(n) for n in range(1, strct[0]+1)]}
    for level in range(1, n_levels):
        level_nests = []
        branch_cntr = 1
        for p_node in nltree_dict['l'+str(level)]:
            for j in range(1, strct[level]+1):
                j_node = p_node + '_' + 'L%dC%d'%(level+1, branch_cntr)
                level_nests.append(j_node)
                branch_cntr += 1
        nltree_dict['l'+str(level+1)] = level_nests
    return nltree_dict

def generate_random_utilities(nltree_dict, n_covariates):
    n_levels = len(nltree_dict) - 1
    choices = nltree_dict['l'+str(n_levels)]
    n_choices = len(choices)
    uts = collections.OrderedDict()
    for choice in range(n_choices):
        uts[choices[choice]] = 'A%d+B%d*X1'%(choice+1, choice+1)
    return uts


### A function to generate random covariate data

In [4]:
def generate_data(nltree_dict, n_cases, 
                  n_covariates, 
                  avail_rate_mean=0.1, 
                  avail_rate_sd=0.05,
                  covs_means_mn=0.5,
                  covs_means_mx=1.5,
                  covs_std_from_means=0.1,
                  use_choice_ids=True):
    
    n_levels = len(nltree_dict) - 1
    choices = nltree_dict['l' + str(n_levels)]
    n_choices = len(choices)
    print('n_choices = %d'%(n_choices))
    def gen_choices(n_avail_choices):
        n_avail_choices = min(n_choices, n_avail_choices)
        if use_choice_ids:
            avail_choices = list(np.sort(np.random.choice(choices, n_avail_choices, replace=False)))
        else:
            avail_choices = list(np.sort(np.random.choice(n_choices, n_avail_choices, replace=False)))
        return avail_choices
    
    loc = avail_rate_mean*n_choices
    scale = avail_rate_sd*n_choices
    n_avail_choices = truncnorm(a=2, b=n_choices, loc=loc, scale=scale).rvs(size=n_cases).astype(int)
    choice_ids_list = list(map(gen_choices, n_avail_choices))
    case_ids = []
    choice_ids = []
    for caseid in range(len(choice_ids_list)):
        sub_l = choice_ids_list[caseid]
        case_ids.extend([caseid]*len(sub_l))
        choice_ids.extend(sub_l)
    ncases_times_n_avail_choices = len(choice_ids)
    covs_means = np.random.uniform(covs_means_mn, covs_means_mx, n_covariates)
    covs = np.random.normal(loc=covs_means, scale=covs_std_from_means*covs_means, 
                            size=[ncases_times_n_avail_choices, n_covariates])
    cov_names = ['X%d'%(i+1) for i in range(n_covariates)]
    d_pd = pd.DataFrame(data=covs, columns = cov_names)
    d_pd['caseid'] = case_ids
    d_pd['choiceid'] = choice_ids
    return d_pd

### Functions to generate random parameter values and constraints

In [5]:
def generate_random_parameters(nltree_dict, n_covariates=None, theta_bins=None, 
                          theta_mn=0.65, theta_mx=0.95):
    n_levels = len(nltree_dict) - 1
    bins = theta_bins
    if bins is None:
        bnds = np.linspace(start=theta_mn, stop=theta_mx, num=n_levels)
        bins = [[bnds[i], None] 
                if i < n_levels-2 else 
                [bnds[i], bnds[i+1]] 
                for i in range(n_levels-1)][::-1]
    params_dict = {}
    logsum_level = []
    for node_name in nltree_dict['l1']:
        logsum = np.random.uniform(bins[0][0], bins[0][1])
        params_dict[node_name] = logsum
        logsum_level.append(logsum)
    logsum_levels = [copy.deepcopy(logsum_level)]
    for level in range(1, n_levels-1):
        logsum_level = []
        for i, p_node in enumerate(nltree_dict['l'+str(level)]):
            for j, c_node in enumerate(nltree_dict['l'+str(level+1)]):
                if p_node in c_node:
                    logsum = np.random.uniform(bins[level][0], logsum_levels[level-1][i])
                    params_dict[c_node] = logsum
                    logsum_level.append(logsum)
            logsum_levels.append(logsum_level)
    n_choices = len(nltree_dict['l' + str(n_levels)])
    n_covariates = n_choices if n_covariates is None else n_covariates
    for i in range(n_covariates):
        params_dict['A%d'%(i+1)] = np.round(np.random.uniform(1.0, 2.0), 2) 
        params_dict['B%d'%(i+1)] = np.round(np.random.uniform(-2.0, -0.1), 2) 
    return params_dict

def get_logsum_constraints(nltree_dict):
    n_levels = len(nltree_dict) - 1
    constraints = {}
    for level in range(1, n_levels):
        nests = nltree_dict['l%d'%(level)]
        for nest in nests:
            constraints[nest] = (0, 1.0)
    return constraints

## Use the functions above to generate the choice tree, utilities, data, and parameters

In [6]:
nltree_dict = generate_nltree(strct=[2,3,4])
n_covariates = 1
n_levels = len(nltree_dict) - 1
choices = nltree_dict['l'+str(n_levels)]
n_choices = len(choices)
utilities_dict = generate_random_utilities(nltree_dict, n_covariates)

In [7]:
utilities_dict

OrderedDict([('L1C1_L2C1_L3C1', 'A1+B1*X1'),
             ('L1C1_L2C1_L3C2', 'A2+B2*X1'),
             ('L1C1_L2C1_L3C3', 'A3+B3*X1'),
             ('L1C1_L2C1_L3C4', 'A4+B4*X1'),
             ('L1C1_L2C2_L3C5', 'A5+B5*X1'),
             ('L1C1_L2C2_L3C6', 'A6+B6*X1'),
             ('L1C1_L2C2_L3C7', 'A7+B7*X1'),
             ('L1C1_L2C2_L3C8', 'A8+B8*X1'),
             ('L1C1_L2C3_L3C9', 'A9+B9*X1'),
             ('L1C1_L2C3_L3C10', 'A10+B10*X1'),
             ('L1C1_L2C3_L3C11', 'A11+B11*X1'),
             ('L1C1_L2C3_L3C12', 'A12+B12*X1'),
             ('L1C2_L2C4_L3C13', 'A13+B13*X1'),
             ('L1C2_L2C4_L3C14', 'A14+B14*X1'),
             ('L1C2_L2C4_L3C15', 'A15+B15*X1'),
             ('L1C2_L2C4_L3C16', 'A16+B16*X1'),
             ('L1C2_L2C5_L3C17', 'A17+B17*X1'),
             ('L1C2_L2C5_L3C18', 'A18+B18*X1'),
             ('L1C2_L2C5_L3C19', 'A19+B19*X1'),
             ('L1C2_L2C5_L3C20', 'A20+B20*X1'),
             ('L1C2_L2C6_L3C21', 'A21+B21*X1'),
             ('L1C2

In [8]:
n_cases = 80000
data = generate_data(nltree_dict, n_cases=n_cases, n_covariates=n_covariates, 
                     avail_rate_mean=0.5, avail_rate_sd=0.1,
                     covs_std_from_means=0.25)

n_choices = 24


In [9]:
data.head()

,X1,caseid,choiceid
0,1.354777,0,L1C1_L2C1_L3C2
1,1.215865,0,L1C1_L2C1_L3C4
2,1.099582,0,L1C1_L2C2_L3C6
3,1.334179,0,L1C1_L2C2_L3C7
4,1.151389,0,L1C1_L2C2_L3C8


In [10]:
true_params_dict = generate_random_parameters(nltree_dict, n_covariates=n_choices, 
                                         theta_mn=0.6, theta_mx=0.9)

### Instantiate an NLFlow model

In [15]:
%load_ext autoreload
%autoreload 2
import DCMFlow_094 as dcm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
m = dcm.NLFlow(nltree_dict=nltree_dict, utilities_dict=utilities_dict)

### Simulate choices & compute loglikelihood

In [17]:
choices_and_ll = m.compute_choices_and_likelihood(data, true_params_dict)
if choices_and_ll is not None:
    simulated_choices, ll = choices_and_ll
    data_with_y = data.copy()
    data_with_y['chosen'] = simulated_choices['chosen'].copy()
    print('Loglikelihood (sum/mean) = %.1f/%.3f'%(-ll, -ll/n_cases))

Loglikelihood (sum/mean) = 195441.6/2.443


### Estimate the model's parameters using the L-BFGS-B optimizer (and compare them with true values)

In [18]:
constraints_dict = get_logsum_constraints(nltree_dict)
m.fit(data_with_y, optimizer='l-bfgs-b',
      true_params_dict=true_params_dict, 
      constraints_dict=constraints_dict)

Digesting the data...

Starting the optimization process using l-bfgs-b
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 195416.468750
  Number of iterations: 84
  Number of functions evaluations: 106
Loglikelihood (Sum/Mean)=195416.5/2.443, T=3s, Cf/Ls/Cn MAPE=7.8/4.7/104.1%, RMSE=0.110/0.039/1.579


### Print estimated and true parameters side by side

In [21]:
m.get_estimated_parameters(true_params_dict=true_params_dict).head()

,param_type,param_name,param_est_val,param_true_val,relative_perc_error
0,Logsum,L1C1,0.752554,0.762773,1.339652
1,Logsum,L1C2,0.810736,0.765592,5.896681
2,Logsum,L1C1_L2C1,0.657672,0.671894,2.116656
3,Logsum,L1C1_L2C2,0.599273,0.617881,3.011443
4,Logsum,L1C1_L2C3,0.684439,0.665747,2.807725


### You could also employ Adam to optimize using stochastic-gradient descent

You could use `m.fit(data_with_y, optimizer='adam', ...)`, however, that would run Adam in batch mode where at each 
iteration the entire data will be used to update the parameters. To run Adam on smaller samples of the data (mini-batches), you will need create and AdamOptimizer object and set its step_size to the desired value, E.g., below I use 5000 cases, I can also set other hyper-parameters as shown below and the left (not listed) assigned to their default values

In [28]:
opt = dcm.AdamOptimizer(n_steps=100, log_every_n_epochs=10, 
                        step_size=10000, learning_rate=2e-4, 
                        patience=2000, epsilon=1e-8,
                        interior_point_penalty_term=0.0005,
                        objective='loglikelihood_sum')

In [29]:
m.fit(data_with_y, optimizer=opt, 
      true_params_dict=true_params_dict,
      constraints_dict=constraints_dict,
      start_over=True)

Digesting the data...

Starting the optimization process using adam


Abbreviations key:
S: Step, E: Epoch, L: Loglikelihood, S/M: Sum/Mean
C: Cost (loglikehood + penalty terms for constrained 
   parameters, printed when constraints are used)
T: Time, Cf/Ls/Cn: Coefficient/Logsum/Constant
MAPE: Mean Absolute Percentage Error (printed when true
      parameters are provided)
RMSE: Root Mean Square Error (printed when true parameters
      are provided)


S/E=8/1, L(S/M)=227298.4/2.841, C(S/M)=227298.5/2.841, T=0s, Cf/Ls/Cn MAPE=100.1/29.5/100.0%, RMSE=1.167/0.208/1.608
S/E=80/10, L(S/M)=224914.0/2.811, C(S/M)=224914.1/2.811, T=6s, Cf/Ls/Cn MAPE=100.9/29.4/100.3%, RMSE=1.158/0.209/1.610
S/E=100/12, L(S/M)=112111.3/2.803, C(S/M)=112111.4/2.803, T=7s, Cf/Ls/Cn MAPE=101.1/29.4/100.4%, RMSE=1.155/0.209/1.611
****************************************************************************************************
STILL CONVERGING: Improvement in Cost of 112310 fell under the Improvement Threshold